# FDM Departure and Arrival
Finding departure and arrival coordinate for each FDM files. Then, create `fdm_location` dataset using the information.

## Import

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import gc
import os

### Constant

In [2]:
# fdm_database_path = '../dataset/database/01_fdm_files.csv'
# fdm_locations_temp = "../dataset/database/fdm_location_write.csv"
# fdm_locations_output = '../dataset/database/02_fdm_locations.csv'

In [2]:
fdm_database_path = '../dataset/database_local/01_fdm_files.csv'
fdm_locations_temp = "../dataset/database_local/fdm_location_write.csv"
fdm_locations_output = '../dataset/database_local/02_fdm_locations.csv'

## Read Data

#### 1. Read FDM Management Data

In [3]:
fdm_files = pd.read_csv(fdm_database_path)
display(fdm_files.head())
print(fdm_files.shape)

,fname,tail_id,fsize_mb,fullpath
0,686200104121532.csv,tail_686_1,68.39,../dataset/fdm/tail_686_1/686200104121532.csv
1,686200104121245.csv,tail_686_1,61.52,../dataset/fdm/tail_686_1/686200104121245.csv
2,686200104121825.csv,tail_686_2,40.71,../dataset/fdm/tail_686_2/686200104121825.csv
3,686200104130429.csv,tail_686_2,42.63,../dataset/fdm/tail_686_2/686200104130429.csv
4,686200104130652.csv,tail_687_1,95.92,../dataset/fdm/tail_687_1/686200104130652.csv


(5, 4)


## Find Departure Coordinate

#### 1. Create Helper Function

In [4]:
def get_first_loc(fdm_df):
    # Given FDM Dataframe, get location (lat, lon) from first row
    if fdm_df.empty:
        return (np.nan, np.nan)
    else:
        return tuple(fdm_df[['lat_rad', 'lon_rad']].iloc[0])

In [5]:
def get_last_loc(fdm_df):
    # Given FDM Dataframe, get location (lat, lon) from last row
    if fdm_df.empty:
        return (np.nan, np.nan)
    else:
        return tuple(fdm_df[['lat_rad', 'lon_rad']].iloc[-1])

In [6]:
def get_departure_loc(fdm):
    # Given fdm dataframe, return the departure long lat and arrival long lat
    plane_in_ground   = fdm[fdm['wow'] == 0]
    plane_off_ground  = fdm[fdm['wow'] == 1]
        
    if plane_in_ground.empty:
        return (np.nan, np.nan)

    if plane_off_ground.empty:
        est_departure_loc = tuple(plane_in_ground[['lat_rad', 'lon_rad']].iloc[0])
        return est_departure_loc
    else:
        first_takeoff_idx = plane_off_ground.index[0]
        departure_df = plane_in_ground.loc[:first_takeoff_idx]
        est_departure_loc = get_first_loc(departure_df)
        return est_departure_loc

In [7]:
def get_arrival_loc(fdm):
    # Given fdm dataframe, return the departure long lat and arrival long lat
    plane_in_ground   = fdm[fdm['wow'] == 0]
    plane_off_ground  = fdm[fdm['wow'] == 1]
        
    if plane_in_ground.empty:
        return (np.nan, np.nan)

    if plane_off_ground.empty:
        return (np.nan, np.nan)
    else:
        first_takeoff_idx = plane_off_ground.index[0]
        arrival_df = plane_in_ground.loc[first_takeoff_idx:]
        est_arrival_loc = get_last_loc(arrival_df)
        return est_arrival_loc

In [8]:
locs = []
def get_loc_from_fdm(fdm_filepath, write=True):
    # Given filepath, return the departure long lat and arrival long lat    
    fdm_df = pd.read_csv(fdm_filepath)
    fdm_df = fdm_df[(fdm_df['lat_rad'] != 0) & (fdm_df['lon_rad'] != 0)]
    dep_lat, dep_lon, arr_lat, arr_lon = get_departure_loc(fdm_df) + get_arrival_loc(fdm_df)
    
    row = (os.path.basename(fdm_filepath), dep_lat, dep_lon, arr_lat, arr_lon)
    locs.append(row)
    
    if write:
        file1 = open(fdm_locations_temp, "a")  # append mode
        file1.write(",".join(str(r) for r in row)  + '\n')
        file1.close()
    
    print(os.path.basename(fdm_filepath), end=': ')
    print((round(dep_lat, 4), round(dep_lon, 4), round(arr_lat, 4), round(arr_lon, 4)), end=' | ')
    print(len(locs), '/', len(fdm_files))
    return (dep_lat, dep_lon, arr_lat, arr_lon)

In [9]:
fpath = fdm_files.fullpath[0]
get_loc_from_fdm(fpath, write=False)

686200104121532.csv: (0.5858, -1.5141, 0.7366, -1.4549) | 1 / 5


(0.585762194239099, -1.51411785826036, 0.736622745011118, -1.4548915730506)

#### 2. Generate Location for All FDM Files

In [10]:
loc_columns = ['dep_lat', 'dep_lon', 'arr_lat', 'arr_lon']
csv_header = ['fname'] + loc_columns

file1 = open(fdm_locations_temp, "w")
L = ','.join(csv_header) + '\n'
file1.writelines(L)
file1.close()

In [11]:
locs = []
loc_tuples = fdm_files[:5].fullpath.map(get_loc_from_fdm)

686200104121532.csv: (0.5858, -1.5141, 0.7366, -1.4549) | 1 / 5
686200104121245.csv: (0.7367, -1.4548, 0.5858, -1.5141) | 2 / 5
686200104121825.csv: (0.7367, -1.4548, 0.6639, -1.4766) | 3 / 5
686200104130429.csv: (0.6639, -1.4765, 0.7366, -1.455) | 4 / 5
686200104130652.csv: (0.7366, -1.455, 0.6177, -1.7034) | 5 / 5


In [12]:
fdm_locations = fdm_files[['fname']].copy()
fdm_locations[loc_columns] = pd.DataFrame(loc_tuples.tolist(), index=fdm_files.index)

In [13]:
fdm_locations.head()

,fname,dep_lat,dep_lon,arr_lat,arr_lon
0,686200104121532.csv,0.585762,-1.514118,0.736623,-1.454892
1,686200104121245.csv,0.736719,-1.454814,0.585786,-1.514111
2,686200104121825.csv,0.736698,-1.454781,0.663881,-1.476550
3,686200104130429.csv,0.663908,-1.476544,0.736575,-1.454951
4,686200104130652.csv,0.736596,-1.454951,0.617748,-1.703400


In [14]:
fdm_locations.to_csv(fdm_locations_output, index=False)